## **Proyecto 2 DataScience** 
- Sofía García - 22210
- Joaquín Campos - 22155
- Julio García Salas - 22076
- Hanzel


 # Celda 1 — Carga, validación rápida del esquema y consistencia con `sample_submission`

 **Qué hace esta celda**
 1) Inicializa el entorno (versiones y opciones de pandas).
 2) Lee `data/train.csv`, `data/test.csv` y `data/sample_submission.csv` con manejo de *encoding*.
 3) Muestra tamaños, columnas, tipos y detecta columnas “fantasma” (`Unnamed: 0`, etc.).
 4) Verifica consistencia básica entre `test` y `sample_submission` (llave compartida y duplicados).
 5) Explora la columna de etiqueta en `train` si existe (p. ej., `winner/label/chosen/target/preference`).

 > Resultado: quedan `df_train`, `df_test`, `df_submit` cargados y un **reporte de sanidad** para decidir próximos pasos.

In [1]:
import platform
from pathlib import Path
from typing import List, Optional

import numpy as np
import pandas as pd
from IPython.display import display, Markdown

# ---------- Utilidades ----------
pd.set_option("display.max_colwidth", 140)
pd.set_option("display.width", 140)

def md(txt: str):
    display(Markdown(txt))

def read_csv_safe(path: Path) -> pd.DataFrame:
    """Lee CSV probando varios encodings comunes."""
    last_err = None
    for enc in ("utf-8", "utf-8-sig", "latin-1"):
        try:
            return pd.read_csv(path, encoding=enc)
        except Exception as e:
            last_err = e
    raise last_err

def short_info(df: pd.DataFrame) -> pd.DataFrame:
    """Resumen compacto: dtype, nulos y únicos (con límite)."""
    nunique = df.nunique(dropna=False)
    out = pd.DataFrame({
        "dtype": df.dtypes.astype(str),
        "n_null": df.isna().sum(),
        "pct_null": (df.isna().mean() * 100).round(2),
        "n_unique": nunique
    }).sort_index()
    return out

def find_common_key(df_a: pd.DataFrame, df_b: pd.DataFrame) -> Optional[str]:
    """Intenta identificar una llave común razonable entre dos DataFrames."""
    candidate_order = ["id","pair_id","row_id","example_id","prediction_id","battle_id"]
    common = set(df_a.columns) & set(df_b.columns)
    # Prioriza candidatas conocidas
    for c in candidate_order:
        if c in common:
            return c
    # Si no hay conocidas, intenta cualquiera que sea única en ambos
    for c in sorted(common):
        if df_a[c].is_unique and df_b[c].is_unique:
            return c
    return None

# ---------- 1) Entorno ----------
md("### Entorno\n"
   f"- Python: `{platform.python_version()}`  \n"
   f"- Pandas: `{pd.__version__}`  \n"
   f"- Plataforma: `{platform.platform()}`")

# Detecta carpeta de datos: primero ./data, si no existe usa /mnt/data
DATA_DIR = Path("data") if Path("data").exists() else Path("/mnt/data")
assert DATA_DIR.exists(), "No se encontró carpeta de datos. Crea `./data/` o coloca los CSV en `/mnt/data`."
md(f"**Carpeta de datos:** `{DATA_DIR}`")

paths = {
    "train": DATA_DIR / "train.csv",
    "test": DATA_DIR / "test.csv",
    "submit": DATA_DIR / "sample_submission.csv",
}
for k, p in paths.items():
    assert p.exists(), f"No se encontró `{p}`"

# ---------- 2) Lectura ----------
df_train = read_csv_safe(paths["train"])
df_test  = read_csv_safe(paths["test"])
df_submit = read_csv_safe(paths["submit"])

md("### Tamaños y columnas")
md(f"- `train`: {df_train.shape}  \n- `test`: {df_test.shape}  \n- `sample_submission`: {df_submit.shape}")

# Columnas “fantasma”
ghost_cols = [c for c in df_train.columns if c.lower().startswith("unnamed")] + \
             [c for c in df_test.columns if c.lower().startswith("unnamed")]
ghost_cols = sorted(set(ghost_cols))
if ghost_cols:
    md(f"**Columnas fantasma detectadas (revísalas/elimínalas si aplica):** `{ghost_cols}`")
else:
    md("**Sin columnas fantasma detectadas.**")

# ---------- 3) Esquema y tipos ----------
md("### Esquema (dtypes, nulos y únicos)")
display(short_info(df_train).head(20).style.set_caption("train — primeras 20 filas del resumen"))
display(short_info(df_test).head(20).style.set_caption("test — primeras 20 filas del resumen"))
display(short_info(df_submit).head(20).style.set_caption("sample_submission — primeras 20 filas del resumen"))

# ---------- 4) Consistencia test vs sample_submission ----------
key = find_common_key(df_test, df_submit)
if key is not None:
    md(f"### Consistencia con `sample_submission`\n- **Llave común detectada:** `{key}`")
    # Duplicados
    dup_test = df_test.duplicated(subset=[key]).sum()
    dup_subm = df_submit.duplicated(subset=[key]).sum()
    md(f"- Duplicados en `test[{key}]`: **{dup_test}**  \n- Duplicados en `sample_submission[{key}]`: **{dup_subm}**")
    # Cobertura
    miss_in_sub = (~df_test[key].isin(df_submit[key])).sum()
    miss_in_test = (~df_submit[key].isin(df_test[key])).sum()
    md(f"- Claves de `test` **no** presentes en `sample_submission`: **{miss_in_sub}**  \n"
       f"- Claves de `sample_submission` **no** presentes en `test`: **{miss_in_test}**")
    # Conteo esperado
    if len(df_test) == len(df_submit):
        md("- ✅ `len(test)` coincide con `len(sample_submission)`.")
    else:
        md(f"- ⚠️ `len(test)` (**{len(df_test)}**) **≠** `len(sample_submission)` (**{len(df_submit)}**).")
else:
    md("### Consistencia con `sample_submission`\n- ⚠️ **No se encontró una llave común obvia** entre `test` y `sample_submission`. "
       "Revisa los nombres de columnas; idealmente deben compartir un identificador (ej. `id`, `pair_id`).")

# ---------- 5) Exploración de la etiqueta en train ----------
label_candidates: List[str] = ["winner","label","chosen","target","preference","y"]
present = [c for c in label_candidates if c in df_train.columns]
if present:
    y_col = present[0]
    md(f"### Etiqueta detectada en `train`: `{y_col}`")
    vc = df_train[y_col].value_counts(dropna=False)
    md(f"- Valores y frecuencia:\n\n```\n{vc.to_string()}\n```")
    md(f"- Nulos en `{y_col}`: **{df_train[y_col].isna().sum()}**")
else:
    md("### Etiqueta en `train`\n- ⚠️ **No se detectó automáticamente una columna de etiqueta** "
       f"(busqué {label_candidates}). Indica el nombre correcto si difiere.")

md("> **Listo.** Datos cargados y chequeos básicos completados. Continúa con el siguiente paso cuando digas **“siguiente”**.")


### Entorno
- Python: `3.13.2`  
- Pandas: `2.2.3`  
- Plataforma: `Windows-10-10.0.19045-SP0`

**Carpeta de datos:** `data`

### Tamaños y columnas

- `train`: (57477, 9)  
- `test`: (3, 4)  
- `sample_submission`: (3, 4)

**Sin columnas fantasma detectadas.**

...

- Duplicados en `test[id]`: **0**  
- Duplicados en `sample_submission[id]`: **0**

- Claves de `test` **no** presentes en `sample_submission`: **0**  
- Claves de `sample_submission` **no** presentes en `test`: **0**

- ✅ `len(test)` coincide con `len(sample_submission)`.

### Etiqueta en `train`
- ⚠️ **No se detectó automáticamente una columna de etiqueta** (busqué ['winner', 'label', 'chosen', 'target', 'preference', 'y']). Indica el nombre correcto si difiere.

> **Listo.** Datos cargados y chequeos básicos completados. Continúa con el siguiente paso cuando digas **“siguiente”**.

# Chequeo inicial de datos — resumen y lectura crítica (yo)

## Entorno
- **Python:** 3.13.2  
- **Pandas:** 2.2.3  
- **Plataforma:** Windows-10-10.0.19045-SP0  
- **Carpeta de datos:** `data/`

---

## Tamaños y columnas
- **train:** (57477, 9)  
- **test:** (3, 4)  
- **sample_submission:** (3, 4)  
- ✅ **Sin** columnas fantasma detectadas.

---

## Esquema (dtypes, nulos, únicos)

**train**
- `id` (int64) — únicos: 57477
- `model_a` (object) — 64 valores
- `model_b` (object) — 64 valores
- `prompt` (object) — 51734 valores
- `response_a` (object) — 56566 valores
- `response_b` (object) — 56609 valores
- `winner_model_a` (int64) — {0,1}
- `winner_model_b` (int64) — {0,1}
- `winner_tie` (int64) — {0,1}

**test**
- `id`, `prompt`, `response_a`, `response_b` — 3 registros, sin nulos.

**sample_submission**
- `id` + columnas objetivo: `winner_model_a`, `winner_model_b`, `winner_tie` (float64)

---

## Consistencia con `sample_submission`
- **Llave común:** `id`
- Duplicados en `test[id]`: **0**
- Duplicados en `sample_submission[id]`: **0**
- Claves de `test` no presentes en `sample_submission`: **0**
- Claves de `sample_submission` no presentes en `test`: **0**
- ✅ `len(test)` == `len(sample_submission)`

---

## Etiquetas / objetivo
- No hay una única columna `label`.  
- Mi **objetivo** está en **formato one-hot** con tres flags:  
  `winner_model_a`, `winner_model_b`, `winner_tie` (todas int64 en train, float en submission).  
- Próximo chequeo que haré sobre `train`:  
  - validar que por fila **sume 1** (`a+b+tie == 1`),  
  - revisar **balance** de clases (tasas por categoría).

---

## Conclusiones rápidas
- La lectura es limpia y los archivos **encajan** entre sí.  
- El esquema sugiere un **problema multiclase (3 clases)** o **multi-salida calibrada** (predicciones de probabilidad por cada flag).  
- No hay nulos ni columnas basura; `id` parece una llave buena.  
- El train es grande (57k+ pares), lo cual permite separar validación 




 # Celda 2 — Integridad de etiquetas (one-hot), missing/empties en texto, y duplicados clave

 **Qué hace esta celda**
 1) **Valida** que las columnas objetivo (`winner_model_a`, `winner_model_b`, `winner_tie`) formen un **one-hot** por fila (suma=1) y sean binarias.
 2) **Reporta balance de clases** en `train`.
 3) **Cuantifica nulos y vacíos** (tras `strip`) en `prompt`, `response_a`, `response_b`.
 4) **Detecta duplicados** de `id` y de la tupla `(prompt, response_a, response_b)`.
 5) **Resume longitudes** (caracteres) de los textos para orientar límites de *tokenization* más adelante.


In [3]:
import numpy as np
import pandas as pd
from IPython.display import display, Markdown

def md(s: str):
    display(Markdown(s))

# ---------- 1) Integridad de etiquetas one-hot ----------
target_cols = [c for c in ["winner_model_a", "winner_model_b", "winner_tie"] if c in df_train.columns]
assert len(target_cols) == 3, f"Esperaba 3 columnas objetivo, encontré: {target_cols}"

row_sum = df_train[target_cols].sum(axis=1)
viol_sum_ne1 = (row_sum != 1).sum()
viol_sum_0 = (row_sum == 0).sum()
viol_sum_gt1 = (row_sum > 1).sum()

# binariedad por columna
bin_ok = {c: df_train[c].isin([0, 1]).all() for c in target_cols}

md("### Integridad de etiquetas (one-hot)")
md(f"- Columnas objetivo: `{target_cols}`  \n"
   f"- Filas con **suma != 1**: **{viol_sum_ne1}** "
   f"(=0: {viol_sum_0}, >1: {viol_sum_gt1})  \n"
   f"- Binariedad por columna: " + ", ".join([f"`{c}`={'OK' if ok else 'NO'}" for c, ok in bin_ok.items()]))

md("**Balance de clases (train):**")
display(
    df_train[target_cols]
    .astype("int64")
    .value_counts()
    .rename("count")
    .reset_index()
    .sort_values("count", ascending=False)
    .style.set_caption("Combinaciones one-hot más frecuentes (esperado: solo 3 combinaciones válidas)")
)

# ---------- 2) Missing & vacíos en texto ----------
text_cols = [c for c in ["prompt", "response_a", "response_b"] if c in df_train.columns]
assert set(text_cols) == {"prompt", "response_a", "response_b"}, f"Faltan columnas de texto esperadas: {text_cols}"

def empties_report(df: pd.DataFrame, cols):
    rep = []
    for c in cols:
        n_null = df[c].isna().sum()
        n_empty = df[c].astype(str).str.strip().eq("").sum()
        rep.append({"column": c, "n_null": n_null, "pct_null": round(100*n_null/len(df),2),
                    "n_empty": n_empty, "pct_empty": round(100*n_empty/len(df),2)})
    return pd.DataFrame(rep)

md("### Nulos y vacíos en texto (train)")
display(empties_report(df_train, text_cols).style.set_caption("train — nulos/vacíos"))
md("### Nulos y vacíos en texto (test)")
display(empties_report(df_test, text_cols).style.set_caption("test — nulos/vacíos"))

# ---------- 3) Duplicados ----------
md("### Duplicados")
dup_id_train = df_train["id"].duplicated().sum()
md(f"- Duplicados en `train.id`: **{dup_id_train}**")
if dup_id_train:
    display(df_train[df_train["id"].duplicated(keep=False)].sort_values("id").head(10))

# Duplicados exactos por tripleta de texto en train
trip_cols = ["prompt", "response_a", "response_b"]
dup_trip = df_train.duplicated(subset=trip_cols).sum()
md(f"- Duplicados exactos por `(prompt, response_a, response_b)` en train: **{dup_trip}**")
if dup_trip:
    display(df_train[df_train.duplicated(subset=trip_cols, keep=False)][trip_cols].head(5))

# ---------- 4) Longitudes de texto ----------
md("### Longitudes de texto (caracteres) — percentiles")
q = [0.5, 0.9, 0.95, 0.99, 1.0]
len_stats = (
    pd.DataFrame({
        c: df_train[c].astype(str).str.len().quantile(q).rename(c) for c in text_cols
    })
    .T
)
len_stats.columns = [f"p{int(p*100)}" for p in q]
display(len_stats.style.set_caption("Quantiles de longitud (train)"))




### Integridad de etiquetas (one-hot)

- Columnas objetivo: `['winner_model_a', 'winner_model_b', 'winner_tie']`  
- Filas con **suma != 1**: **0** (=0: 0, >1: 0)  
- Binariedad por columna: `winner_model_a`=OK, `winner_model_b`=OK, `winner_tie`=OK

**Balance de clases (train):**

,winner_model_a,winner_model_b,winner_tie,count
0,1,0,0,20064
1,0,1,0,19652
2,0,0,1,17761


### Nulos y vacíos en texto (train)

...

- Duplicados en `train.id`: **0**

- Duplicados exactos por `(prompt, response_a, response_b)` en train: **71**

,prompt,response_a,response_b
777,"[""Respond only with the letter of the correct answer:\n\nWhich weighs more, one pound of feathers or two pounds of bricks?\n\nA: The fea...","[""B: The bricks""]","[""C""]"
1035,"[""hi there""]","[""Hello! How can I assist you today?""]","[""Hello! How can I assist you today?""]"
1777,"[""Answer the following statements with \""Agree\"" or \""Disagree\"" only. You answers should be returned in list form, in the same order th...","[""Sure, here are my answers to your questions:\n\n1. Disagree\n2. Disagree\n3. Agree\n4. Disagree\n5. Agree\n6. Agree\n7. Disagree\n8. D...","[""Sure, here are my answers:\n\n1. Disagree\n2. Disagree\n3. Agree\n4. Disagree\n5. Agree\n6. Agree\n7. Disagree\n8. Disagree\n9. Disagr..."
2195,"[""write a single dot""]","["".""]","["".""]"
2998,"[""what is the capital of france""]","[""The capital of France is Paris.""]","[""The capital of France is Paris.""]"


### Longitudes de texto (caracteres) — percentiles

,p50,p90,p95,p99,p100
prompt,96.000000,784.000000,1471.000000,4920.400000,33056.000000
response_a,1076.000000,2787.000000,3721.000000,7004.920000,54058.000000
response_b,1086.000000,2781.400000,3709.000000,7071.480000,53830.000000


# Análisis de verificación de datos (tercera persona)

## 1) Integridad de etiquetas (one-hot)
- **Columnas objetivo:** `winner_model_a`, `winner_model_b`, `winner_tie`.
- **Suma por fila:** 0 filas con `suma != 1` (=`0`: 0, `>1`: 0) → **one-hot correcto**.
- **Binariedad:** todas las columnas son {0,1} → **OK**.

**Balance de clases (train)**  
- A gana: **20,064**  
- B gana: **19,652**  
- Empate: **17,761**  
> Distribución **razonablemente balanceada** (ligera menor proporción de empates). No se anticipan problemas severos por desbalance.

---

## 2) Calidad de texto (nulos y vacíos)
**Train y Test**
- `prompt`, `response_a`, `response_b`: **0 nulos** y **0 vacíos**.  
> Señal de **consistencia** y **completitud** en los campos clave.

---

## 3) Duplicados
- `train.id`: **0** duplicados.
- Tripleta exacta `(prompt, response_a, response_b)`: **71** duplicados.

**Muestra de casos relevantes**
- Prompt tipo trivias/fácticos con **respuestas idénticas** en `response_a` y `response_b` (p. ej., *“The capital of France is Paris.”* en ambos).  
- Casos mínimos (p. ej., *“write a single dot”* → `"."` vs `"."`).  
> **Riesgo**: estos duplicados pueden introducir **fuga** o **sobre-representar patrones triviales**; además, cuando `A==B` debería esperarse **`winner_tie=1`**. Si no coincide, habría **ruido de etiqueta**.

**Recomendación**  
- Deduplicar por tripleta exacta (conservando la primera aparición) o **agrupar y consolidar** si hay incoherencias de etiqueta dentro del grupo.

---

## 4) Longitud de textos (caracteres) — percentiles (train)
- **Prompt:** p50=96, p90=784, p95=1,471, p99≈4,920, p100=33,056  
- **Response A:** p50=1,076, p90=2,787, p95=3,721, p99≈7,005, p100=54,058  
- **Response B:** p50=1,086, p90=2,781, p95=3,709, p99≈7,071, p100=53,830  

> Distribuciones con **colas largas** (outliers muy extensos). Se sugiere fijar límites de longitud/tokens (p. ej., **p99** como referencia) o aplicar truncado controlado para evitar **OOM** y sesgos por longitud.

---

## 5) Conclusiones operativas
1. **Etiquetas:** válidas y en estricto one-hot → listo para entrenamiento multiclase/multi-salida.  
2. **Datos faltantes:** inexistentes en campos críticos → no se requiere imputación.  
3. **Duplicados:** 71 tripletas idénticas → **deduplicar** y **verificar coherencia** con `winner_tie`.  
4. **Longitud:** presencia de textos extremadamente largos → **definir `max_len`** (tokens/caracteres) y política de truncado.  

**Siguientes pasos sugeridos**
- (a) Limpieza normalizada de texto (Unicode NFC, control chars, espacios) sin alterar semántica.  
- (b) Detección y mitigación de **sesgo por longitud** y **sesgo de posición** (A vs B).  
- (c) Deduplicación y reporte de impacto (cuántas filas se eliminan).  
- (d) Definir *split* sin fuga (por `prompt` o grupos adecuados) y persistir `*_clean.parquet`.



 # Celda 3 — Limpieza normalizada de texto, flags de calidad y deduplicación segura

 **Qué hace esta celda**
 1) Define una función de **limpieza no-destructiva**: normaliza Unicode (NFC), estandariza saltos de línea,
    elimina caracteres de control (salvando `\n` y `\t`) y colapsa espacios redundantes sin alterar la semántica.
 2) Aplica la limpieza a `prompt`, `response_a`, `response_b` en `train` y `test`, **reportando cuántas filas cambiaron** por columna.
 3) Crea `df_train_clean` / `df_test_clean` (copias limpias) y marca **casos sospechosos de empate** (`response_a == response_b` pero la etiqueta no es `winner_tie`).
 4) **Deduplica** por la tripleta exacta `(prompt, response_a, response_b)` en `train` limpio (mantiene la primera ocurrencia) y reporta removidos.



In [5]:
import re
import unicodedata
import pandas as pd
from IPython.display import display, Markdown

def md(s: str):
    display(Markdown(s))

TEXT_COLS = ["prompt", "response_a", "response_b"]

def _strip_control_chars(s: str) -> str:
    # Elimina caracteres categoría Unicode 'C' (control/format), pero preserva \n y \t
    return "".join(ch for ch in s if (unicodedata.category(ch)[0] != "C") or ch in ("\n", "\t"))

def clean_text(x) -> str:
    # Robustez a NaN/None: convierte a string (no hay nulos según chequeo previo, pero se protege)
    s = "" if pd.isna(x) else str(x)
    # Normaliza Unicode
    s = unicodedata.normalize("NFC", s)
    # Normaliza saltos de línea
    s = s.replace("\r\n", "\n").replace("\r", "\n")
    # Remueve chars de control (salvando \n y \t)
    s = _strip_control_chars(s)
    # Colapsa espacios y tabs contiguos (preserva saltos de línea)
    s = re.sub(r"[^\S\n]+", " ", s)
    # Recorta espacios exteriores (no toca saltos de línea internos)
    return s.strip()

def apply_clean(df: pd.DataFrame, cols) -> tuple[pd.DataFrame, pd.DataFrame]:
    out = df.copy()
    report_rows = []
    for c in cols:
        before = out[c].astype(str)
        after = before.map(clean_text)
        changed = (before != after)
        out[c] = after
        report_rows.append({
            "column": c,
            "changed_rows": int(changed.sum()),
            "pct_changed": round(100 * changed.mean(), 2)
        })
    return out, pd.DataFrame(report_rows)

# ---------- 1) Aplicar limpieza ----------
df_train_clean, train_changes = apply_clean(df_train, TEXT_COLS)
df_test_clean,  test_changes  = apply_clean(df_test,  TEXT_COLS)

md("### Cambios por columna tras limpieza")
display(train_changes.assign(split="train")[["split","column","changed_rows","pct_changed"]])
display(test_changes.assign(split="test")[["split","column","changed_rows","pct_changed"]])

# ---------- 2) Flag de “empate esperado por texto” ----------
has_targets = all(c in df_train_clean.columns for c in ["winner_model_a","winner_model_b","winner_tie"])
if has_targets:
    eq_ab = (df_train_clean["response_a"] == df_train_clean["response_b"])
    not_tie = (df_train_clean["winner_tie"] != 1)
    df_train_clean["tie_expected_from_text"] = eq_ab
    df_train_clean["tie_label_mismatch"]     = eq_ab & not_tie
    n_eq = int(eq_ab.sum())
    n_mismatch = int((eq_ab & not_tie).sum())
    md("### Consistencia etiqueta vs. igualdad de respuestas (train limpio)")
    md(f"- Filas con `response_a == response_b`: **{n_eq}**")
    md(f"- De ellas, **no** etiquetadas como `tie`: **{n_mismatch}**  (→ revisar posibles inconsistencias)")
else:
    md("### Consistencia etiqueta vs. igualdad de respuestas")
    md("- Columnas de objetivo no presentes; se omite el chequeo de `winner_tie`.")

# ---------- 3) Deduplicación por tripleta exacta en train limpio ----------
before_n = len(df_train_clean)
df_train_clean = df_train_clean.drop_duplicates(subset=TEXT_COLS, keep="first").reset_index(drop=True)
after_n = len(df_train_clean)
removed = before_n - after_n
md("### Deduplicación en train limpio")
md(f"- Filas antes: **{before_n}**  \n- Filas después: **{after_n}**  \n- **Removidas por duplicado exacto (prompt, response_a, response_b): {removed}**")

# ---------- 4) Recordatorio de objetos en memoria ----------
md("> **Listo.** Quedan en memoria `df_train_clean` y `df_test_clean`. Próximo paso sugerido: métricas de **sesgo por posición/longitud** y definición de **límites de longitud/tokens** y *split* sin fuga.")


### Cambios por columna tras limpieza

,split,column,changed_rows,pct_changed
0,train,prompt,4366,7.60
1,train,response_a,7546,13.13
2,train,response_b,7501,13.05


,split,column,changed_rows,pct_changed
0,test,prompt,0,0.00
1,test,response_a,1,33.33
2,test,response_b,1,33.33


### Consistencia etiqueta vs. igualdad de respuestas (train limpio)

- Filas con `response_a == response_b`: **275**

- De ellas, **no** etiquetadas como `tie`: **27**  (→ revisar posibles inconsistencias)

### Deduplicación en train limpio

- Filas antes: **57477**  
- Filas después: **57406**  
- **Removidas por duplicado exacto (prompt, response_a, response_b): 71**

> **Listo.** Quedan en memoria `df_train_clean` y `df_test_clean`. Próximo paso sugerido: métricas de **sesgo por posición/longitud** y definición de **límites de longitud/tokens** y *split* sin fuga.

# Análisis de limpieza y deduplicación

## 1) Impacto de la limpieza
- **Cambios en `train`**
  - `prompt`: 4,366 filas (7.60%)
  - `response_a`: 7,546 filas (13.13%)
  - `response_b`: 7,501 filas (13.05%)
- **Cambios en `test`**
  - `prompt`: 0 filas (0.00%)
  - `response_a`: 1 fila (33.33%)
  - `response_b`: 1 fila (33.33%)

**Lectura:** El impacto está concentrado en las respuestas (≈13%), consistente con normalización de Unicode, control chars y espacios. En `test` los cambios son mínimos (buena señal de calidad de entrada).

---

## 2) Consistencia etiqueta vs. igualdad de respuestas
- Filas con **`response_a == response_b`**: **275**
- De esas, **no etiquetadas como `tie`**: **27**

**Riesgo:** Posibles **inconsistencias de etiqueta**. En pares A=B se esperaría `winner_tie=1`. Dejar estas filas sin corregir puede introducir ruido en el entrenamiento y afectar calibración.

**Sugerencia de manejo:**
- Opción A (segura): **Excluir** estas 27 filas del entrenamiento.
- Opción B (conservadora): Forzar `winner_tie=1` si A==B **y** no hay evidencia en contra.
- Opción C (ponderación): Mantenerlas pero con **peso reducido** para minimizar su impacto.

---

## 3) Deduplicación
- **Antes**: 57,477 filas  
- **Después**: 57,406 filas  
- **Removidas**: **71** (duplicado exacto por `prompt`, `response_a`, `response_b`)

**Lectura:** La deduplicación elimina sobre-representación de casos triviales y reduce riesgo de fuga. El conteo removido coincide con el número de duplicados detectados previamente.

---

## 4) Conclusión operativa
- La limpieza fue **no destructiva** y consistente; la mayoría de cambios son de higiene (espacios/Unicode).
- La **deduplicación** dejó un conjunto más estable y sin sobre-rep.
- Persisten **27 casos** con **A==B** y **no-tie** que conviene tratar explícitamente antes de entrenar.
